In [130]:
from pymongo import MongoClient
from pathlib import Path
import os
from pprint import pprint
import nltk
import matplotlib.pyplot as plt
import pprint
import pywt

import os
import struct
import matplotlib
from scipy import signal
import scipy.fftpack
import numpy as np
import sys
import pymongo
import json

import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
%matplotlib inline

In [131]:
client = MongoClient('mongodb://localhost:27017')
testDb = client.testDb

patients = testDb.patients
sessions = testDb.sessions
reports = testDb.reports

In [132]:
def fileName(filePath):
    return (os.path.splitext(os.path.split(filePath)[1]))[0]

def fileExt(filePath):
    try:
        return (os.path.splitext(os.path.split(filePath)[1]))[1]
    except:
        return "Error"

In [133]:
class channel(object):
    def __init__(self,edfFile,label=None,transducer=None,dimensions=None,
                physMin=None,physMax=None,digMin=None,digMax=None,
                 preFilter=None,sampleRate=None,notes=None,channelIndex=None,byteLength=None):
        self.edfFile = edfFile
        self.label = str(label)
        self.sampleRate = int(sampleRate)
        self.transducer = str(transducer)
        self.dimensions = dimensions
        self.physMin = physMin
        self.physMax = physMax
        self.digMin = digMin
        self.digMax = digMax
        self.preFilter = preFilter
        self.notes  = notes
        self.channelIndex = int(channelIndex)
        self.byteLength = byteLength
        #self.indexStart = indexStart
    
    def __str__(self):
        if self.label==None:
            return "None"
        else:
            returnString = ("Label: "+str(self.label) + "\nSample Rate"+str(self.sampleRate)+
            "\nTransducer: "+str(self.transducer)+"\nDimensions: "+str(self.dimensions)+
            "\nphysMin: "+str(self.physMin)+"\nphysMax: "+str(self.physMax)+
            "\ndigMin: "+str(self.digMin) +"\ndigMax: "+str(self.digMax)+ "\npreFilter: "+str(self.preFilter)+
            "\nNotes: "+str(self.notes) + "\nChannelIndex: "+str(self.channelIndex) + "\nbyteLength: "+str(self.byteLength))
        return returnString
    
    def channelData(self,start,duration,channelNumber = 0,sensitivity = 1):
        '''
        Returns data points for desired area
        start: seconds from start of record
        duration: time  of record desired in seconds
        ##########This might be easier to do within original edf class
        Need to change to accomdate FLOATING POINTS !!!!!!!!!!!
        '''
        toReturn = []
        startByte = float(start) * self.sampleRate * 2
        '''
        print("TYPE START: ",type(start))
        print("channel.channelData Sample Rate: ",type(self.sampleRate))
        print("Start: ",start)
        print("START BYTE FROM CHANNEL DATA: ",startByte," Start: ",start," SampleRate: ",self.sampleRate)
        '''
        seekLocation = int(self.channelIndex + int(startByte))  #need to fix this --------------
        #print('Seek Location: ',seekLocation)
        numberOfBytes = int(self.sampleRate) * float(duration) #* 2
        try:
            numberOfBytes = int(numberOfBytes)
        except:
            return "ERROR in numberOfBytes"
        
        self.edfFile.edfRawData.seek(seekLocation)
        for i in range(numberOfBytes):
            try:
                value = struct.unpack('h',self.edfFile.edfRawData.read(2))[0]
                value = value*sensitivity    #Changed this from np.array(value)*sensitivity
                value -= channelNumber*20*sensitivity
            except:
                value = 'E'
            toReturn.append(value)
            #toReturn.append(struct.unpack('h',self.edfFile.edfRawData.read(2))) 
        return toReturn

    def allChannelData(self):
        toReturn = []
        self.edfFile.edfRawData.seek(self.channelIndex)
        for i in range(int(self.byteLength/2)):
            toReturn.append(struct.unpack('h',self.edfFile.edfRawData.read(2)))
        return toReturn
        
    def getLabel(self):
        return str(self.label)
    
    def getSampleRate(self):
        return self.sampleRate
    
    def getChannelIndex(self):
        return self.channelIndex
    
    def getTransducer(self):
        return self.transducer

    def getDimensions(self):
        return self.dimensions
    
    def getPhysMin(self):
        return float(self.physMin)
    
    def getPhysMax(self):
        return float(self.physMax)
    
    def getDigMin(self):
        return float(self.digMin)
    
    def getDigMax(self):
        return float(self.digMax)
    
    def getPreFilter(self):
        return self.preFilter
    
    def getNotes(self):
        return self.notes
    
    def getByteLength(self):
        return self.byteLength
    
class edfData(object):
    def __init__(self,fileId):
        self.fileId=fileId
        
    def __str__(self):
        return ("version: " + str(self.version) + "\nPatient Id: "+str(self.patientId) 
                + "\nrecordId" + str(self.recordId) + "\nStart Date: " + str(self.startTime)
                + "\nStart Time: " + str(self.startTime)
                + "\nheader Bytes: " + str(self.headerBytes) +"\nReserve Bytes: " +str(self.reservedBytes)
                + "\nNum Records :" + str(self.nRecords) + "\nNum Signals :" + str(self.nSignals) 
                + "\nDuration: "+ str(self.duration) + "\nFile Size: "+str(self.fileSize))
                
                
    def loadFile(self,filepath):
        self.filePath = filepath
        self.fileSize = os.path.getsize(filepath)
        self.edfRawData = open(filepath,'rb')
        
    def readBitsTest(self):
        self.edfRawData.seek(0)
        bits = self.edfRawData.read(10000)
        self.edfRawData.seek(0)
        print(self.edfRawData.read(10))
        print("Some Bits: "+str(bits))
        return bits
    
    def readSignalHeaderBits(self,bitsIn,nSignals,nBits):
            returnArray = []
            for x in range(nSignals):
                returnArray.append(bitsIn.read(nBits))
            return returnArray
        
    def returnChannelObjects(self):
        return self.channels
    
    def parseEdf(self):
        '''
            This function parses the edf file getting the values of 
            the header fields. It also creates a channel object with 
            information about each channel, including the eeg tracing data from each channel.
        '''
        #Header Fields
        self.version = str(self.edfRawData.read(8))
        self.patientId = str(self.edfRawData.read(80))
        self.recordId = str(self.edfRawData.read(80))
        self.startDate = str(self.edfRawData.read(8))
        self.startTime = str(self.edfRawData.read(8))
        self.headerBytes = int(self.edfRawData.read(8))
        self.reservedBytes = str(self.edfRawData.read(44))
        self.nRecords = int(self.edfRawData.read(8))
        self.duration = float(self.edfRawData.read(8))
        self.nSignals = int(self.edfRawData.read(4))
        
        #Create channel object for each channel
        self.channels = {}
        channelFields = ['labels','transducer','dimensions','physMin','physMax','digMin','digMax',
                        'preFilter','sampleRate','reservedNotes']
        self.channels['labels'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,16)
        self.channels['transducer'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,80)
        self.channels['dimensions'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,8)
        self.channels['physMin'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,8)
        self.channels['physMax'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,8)
        self.channels['digMin'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,8)
        self.channels['digMax'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,8)
        self.channels['preFilter'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,80)
        self.channels['sampleRate'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,8)
        self.channels['reservedNotes'] = self.readSignalHeaderBits(self.edfRawData,self.nSignals,32)
        
        self.channelObjects = []
        self.sampleRate = int(self.channels['sampleRate'][0])
        channelDataStart = self.headerBytes
        
    
        #This loop creates and adds channel objects to self.channelObjects
        for i in range(int(self.nSignals)):
            
            channelValues ={'filepath':self.filePath}
            byteLength = int(self.channels['sampleRate'][i])*self.nRecords     #*2
            channelIndex = channelDataStart
            channelDataStart +=byteLength
            
            for j in channelFields:
                try:
                    channelValues[j]=(self.channels[j][i])
                except:
                    channelValues[j] = None
            self.channelObjects.append(
                channel(self,channelValues['labels'],channelValues['transducer'],
                        channelValues['dimensions'],channelValues['physMin'],channelValues['physMax'],
                        channelValues['digMin'],channelValues['digMax'],channelValues['preFilter'],
                        channelValues['sampleRate'],channelValues['reservedNotes'],
                        channelDataStart,byteLength))  #Changed this channelIndex to channelDataStart
    
    
        self.channelSampleRates = channelValues['sampleRate'] 
        self.channelDigMax = channelValues['digMax']
        self.channelDigMin = channelValues['digMin']
        self.channelPhysMax = channelValues['physMax']
        self.channelPhysMin = channelValues['physMin']
            
    def data(self,start,duration):
        sampleData = []
        j = 0
        for x in self.channelObjects:
            sampleData.append([x.getLabel(),x.channelData(start,duration)])
            j+=1
        return sampleData
    
    def dataDictionary(self,start,duration):
        sampleData = {}
        j = 0
        #Returns dictionary with key being channel label, values being a dict [data,sampleRate,digMax,digMin,physMax,physMin]
        for x in self.channelObjects:
            sampleData[x.getLabel()]={'data':x.channelData(start,duration),'sampleRate':x.getSampleRate(),
                                     'digMax':x.getDigMax(),'digMin':x.getDigMin(),'physMin':x.getPhysMin(),
                                     'physMax':x.getPhysMax()}
            #print(x)
            j+=1
        return sampleData
    
    def dataForGraph(self,start,duration,sensitivity=1):
        sampleData = []
        j=0
        for x in self.channelObjects:
            sampleData.append([x.getLabel(),x.channelData(start,duration,j,sensitivity)])
            j+=1
        return sampleData
    
    def allData(self):
        '''
        Return[sampleRate,{channelData}]
        '''
        channelData = {}
        for x in self.channelObjects:
            channelData[str(x.getLabel())]=x.allChannelData()
        return[int(self.sampleRate),channelData]
            
            
    
    def patientId(self):
        return self.patientId
    
    def recordId(self):
        return self.recordId
    
    def startDate(self):
        return self.startDate
    
    def startTime(self):
        return self.startTime
    
    def headerBytes(self):
        return self.headerBytes
    
    def reservedBytes(self):
        return self.reservedBytes
    
    def duration(self):
        return self.duration
    
    def nRecords(self):
        return self.nRecords
    
    def numSignals(self):
        return self.nSignals
    
    def getSampleRate(self):
        return int(self.sampleRate)

    def channelLabels(self):
        labels = []
        for channel in self.channelObjects:
            labels.append(channel.label())
        return labels
    
    def totalSamples(self):
        duration = int(self.sampleRate) * int(self.nRecords)
        return duration
    
    def testChannelPrint(self):
        print(self.channelObjects[2])

In [134]:
'''
Plotting Functions
'''
    
    
def plotEDF(edfObject,montageArray,startPlot,endPlot,maxIndexMarkers=False):
    edfDataDictionary = edfObject.dataDictionary(startPlot,endPlot)
    dataDictionaryKeys = list(edfDataDictionary.keys())
    
    for i,x in enumerate(montageArray):
        channel1 = x[0]
        channel2 = x[1]
        channel1Label = dataDictionaryKeys[channel1]
        channel2Label = dataDictionaryKeys[channel2]
        
        channel1DigMax = edfDataDictionary[channel1Label]['digMax']
        channel1PhysMax = edfDataDictionary[channel1Label]['physMax']
        channel2DigMax = edfDataDictionary[channel1Label]['digMax']
        channel2PhysMax = edfDataDictionary[channel1Label]['physMax']
        
        channel1ScaleFactor = channel1PhysMax/channel1DigMax
        channel2ScaleFactor = channel2PhysMax/channel2DigMax
        
        input1Data = np.array(edfDataDictionary[channel1Label]['data']) * channel1ScaleFactor
        input2Data = np.array(edfDataDictionary[channel2Label]['data']) * channel2ScaleFactor
        
        plotData = input1Data - input2Data

        scaledAndMoved = plotData +(-i*600)
        channelAverageHeight = np.average(scaledAndMoved)
        
        channelsInMontage = len(montageArray)
        vLineHeight = 1/channelsInMontage
        
        if maxIndexMarkers == True:
            channelMax = max(plotData)
            maxIndex = list(plotData).index(channelMax)
            lineStart = 1-((i+2)*(vLineHeight))
            lineEnd = 1-(i*vLineHeight)
            plt.text(maxIndex,channelAverageHeight+60,("Index: "+str(maxIndex)+" Value: "+str(channelMax)))
            #plt.axes().axvline(maxIndex,i*vLineHeight,(i+1)*vLineHeight).set_color('green') This one is if plots aren't started from the bottom
            plt.axes().axvline(maxIndex,lineStart,lineEnd).set_color('green')

        #plt.text(maxIndex,channelAverageHeight+60,("Channel: ",str(x)," Max Index: ",str(maxIndex)))
        plt.text(0,channelAverageHeight+60,(str(channel1Label)+" - "+str(channel2Label)))
        plt.plot(scaledAndMoved)

    plt.show()
    

In [135]:
'''
Global Variables
'''

#Longitudinal Bipolar Montage array
longBipolarMontage = [[0,10],[10,12],[12,14],[14,8],[0,2],[2,4],[4,6],[6,8],
                      [18,19],[19,20],[0,3],[3,4],[4,7],[7,9],[1,11],[11,13],[13,15],[15,9]] 

#adjacentElectrodeArray is an array of arrays where each of the entries is a quadrant LAQ,LPQ,RAQ,RPQ
adjacentElectrodeArray = [[0,1,4,5],[2,6,3,7],[10,11,14,15],[12,11,13,17]] 

#Contains Many Spikes
trainingFile1 ='/Users/taylorsomma/Desktop/capstone/v1.2.0/eval/01_tcp_ar/00009842/s001_2013_04_01/00009842_s001_t000.edf'

trainingEDF1 = edfData(1)
trainingEDF1.loadFile(trainingFile1)
trainingEDF1.parseEdf()
print(trainingEDF1)


version: b'0       '
Patient Id: b'00009842                                                                        '
recordIdb'Startdate 01-APR-2013 00009842_s001                                             '
Start Date: b'14.17.46'
Start Time: b'14.17.46'
header Bytes: 9472
Reserve Bytes: b'EDF                                         '
Num Records :1568
Num Signals :36
Duration: 1.0
File Size: 25890880


In [60]:
keepPlotting = 1
while keepPlotting != -1:
    keepPlotting = input()
    plotStart = int(keepPlotting)
    plotEnd = int(plotStart+10)
    plt.figure(figsize=(20,10))
    plotEDF(trainingEDF1,longBipolarMontage,plotStart,plotEnd,True)

0


NameError: name 'x' is not defined

In [61]:
'''
Signal Processing Functions
'''

def getChangeArray(inputArray):
    newArray =[]
    for i in range(len(inputArray)):
        if i==0:
            pass
        newArray.append(inputArray[i]-inputArray[i-1])
    return newArray

def movingAverage(inputArray,avgNum):
    newArray = []
    for i in range(len(inputArray)):
        if i<avgNum:
            pass
        newArray.append(np.average(inputArray[i-(avgNum):i]))
    return newArray

def returnVariation(eegData,width,sampleRate):
    #Looks for greatest change over the specified width in the signal
    #width is samples to 
    v1 = eegData[0]
    totalChange = 0 
    maxChange = 0
    maxIndex = 0
    dataLength = len(eegData)
    for i in range(dataLength-width):
        currentChange = 0
        for j in range(width-1):
            currentChange += eegData[j+i]-eegData[j+i+1]
             
        if abs(currentChange) > abs(maxChange):
            maxChange = currentChange
            maxIndex = i
        #print(currentChange)
        currentChange = 0
        
    msTime = maxIndex/sampleRate
    #print("Max Change: ",maxChange," Max Index: ",maxIndex," Max Time: ",msTime)
    return [maxChange,maxIndex,msTime]

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff,btype='high', analog = False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order = 5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def range_filter(data, low, high, fs, order = 5):
    dataLow = butter_lowpass_filter(data,high,fs,order)
    dataReturn = butter_highpass_filter(dataLow,low,fs,order)
    return dataReturn
    

def zeroAverage1(data):
    average = np.mean(np.absolute(data))
    returnData = []
    for x in data:
        returnData.append(((x-average)/average)**2)
    return returnData

def normalizeSignal(data):
    maxValue = max(data)
    minValue = min(data)
    valueRange = maxValue-minValue
    returnArray = []
    for x in data:
        returnArray.append(x/valueRange)
    return returnArray
        

In [141]:
def lookForPossibleChannelSpikes(edfObject,montageArray,stepSize,searchSize,ampThreshold,ptpThreshold,durationToCheck,filterTag=False,loFilter=1,highFilter=40):
    '''
    data should already be scaled 
    stepSize: in mSec
    searchSize: in seconds
    ampThreshold: ratio of max to average
    returns: Dictionary {MontageChannelIndex:[spikeIndices]}
    '''
    
    edfDataDictionary = edfObject.dataDictionary(0,durationToCheck)
    dataDictionaryKeys = list(edfDataDictionary.keys())
    possibleSpikeDictionary = {}
    for i,channel in enumerate(montageArray):
        channel1Index = channel[0]
        channel2Index = channel[1]
        channel1Label = dataDictionaryKeys[channel1Index]
        channel2Label = dataDictionaryKeys[channel2Index]
        channel1 = edfDataDictionary[channel1Label]
        channel2= edfDataDictionary[channel2Label]
        
        channel1DigMax = channel1['digMax']
        channel1PhysMax = channel1['physMax']
        channel2DigMax = channel2['digMax']
        channel2PhysMax = channel2['physMax']
        sampleRate = channel1['sampleRate']
        
        channel1ScaleFactor = channel1PhysMax/channel1DigMax
        channel2ScaleFactor = channel2PhysMax/channel2DigMax
        
        input1Data = np.array(edfDataDictionary[channel1Label]['data']) * channel1ScaleFactor
        input2Data = np.array(edfDataDictionary[channel2Label]['data']) * channel2ScaleFactor
        
        channelData = input1Data - input2Data    
        
        sampleSearchSize = searchSize * sampleRate  #Size to search in each iteration
        stepSampleSize = int((stepSize/1000)*sampleRate)  #samples to move during each step
        totalSamples = durationToCheck * sampleRate  #Total Number of samples to check in data file
        numberOfSteps = int(totalSamples/stepSampleSize) #Total number of steps to take
        
        print("SampleSearchSize: ",sampleSearchSize,"\nstepSampleSize: ",stepSampleSize,
             "\ntotalSamples: ",totalSamples,"\nnumberOfSteps: ",numberOfSteps)
    
        possibleSpikeArray = []
        
        for x in range(numberOfSteps):
            startIndex = x*stepSampleSize
            endIndex = (x*stepSampleSize)+sampleSearchSize
            segmentToCheck = channelData[startIndex:endIndex]
            segmentMax = np.amax(np.absolute(segmentToCheck))
            segmentPTP = np.ptp(segmentToCheck)
            segmentAverage = abs(np.average(segmentToCheck))
            print("StartIndex: ",startIndex,"\nEndIndex: ",endIndex)
            freqs, pxx = scipy.signal.periodogram(segmentToCheck, sampleRate)
            amplitudeThreshold = segmentAverage * ampThreshold
            ptpThreshold = segmentAverage * ptpThreshold
            #print("AmplitudeThreshold: ",amplitudeThreshold,"\nSegment Max: ",segmentMax,"\nSegmentPTP: ",segmentPTP,
            #     "\nSegmentAverage: ",segmentAverage)
            if segmentMax > amplitudeThreshold and segmentPTP > amplitudeThreshold:
                try:
                    possibleSegmentIndex = list(segmentToCheck).index(segmentMax)
                except:
                    possibleSegmentIndex = list(segmentToCheck).index(0-segmentMax)
                    
                channelSpikeIndex = startIndex + possibleSegmentIndex
                if channelSpikeIndex not in possibleSpikeArray: 
                    possibleSpikeArray.append(channelSpikeIndex)
        
        possibleSpikeDictionary[i] = possibleSpikeArray
    return possibleSpikeDictionary
                    
                    
    


def checkQuadrants(indexDict,adjacencyArray,edfObject,latencyAllowed):
    '''
    adjacencyArray: list of lists containing channel indices in quadrant
    indexDict: {channelNumber:[spikeCandidates]}
    latencyAllowed: latency in mSec allowed
    '''
    spikeCandidatesFinal = {}
    for quadrant in adjacencyArray:
        for channelIndex in quadrant:
            for spikeIndex in indexDict[channelIndex]:
                

def removeIsolatedSpikes(edfObject,,sampleRate,latencyAllowed):
    '''
    indexArray: Array of indices of possible spikes in a quadrant
    sampleRate: sampleRate of data
    latencyAllowed: distance between possible spikes in msec
    '''
    returnArray = []
    indexLength = len(indexArray)
    removed = 0
    addThisSpike = True
    for i, x in enumerate(indexArray):
        if i==0:
            returnArray.append(x)
            pass
        for y in returnArray:
            if abs(x-y)<minDistanceBetweenSpikes:
                addThisSpike = False
        if addThisSpike==True:
            returnArray.append(x)
        else:
            addThisSpike=True
            removed+=1
            

def removeCloseSpikes(indexArray,edfObject,minDistanceBetweenSpikes):
    '''
    indexArray: Array of indices of possible spikes in a quadrant
    sampleRate: sampleRate of data
    latencyAllowed: distance between possible spikes in msec
    '''
    edfSampleRate = edfObject.getSampleRate()
    indexArraySorted = indexArray.sort()
    latencySamplesAllowed = (minDistanceBetweenSpikes/1000) * edfSampleRate
    returnArray = []
    indexLength = len(indexArray)
    removed = 0
    for i, x in enumerate(indexArray):
        sampleDistance = 1
        if i <= 1:
            returnArray.append(x)
        else:
            returnLength = len(returnArray)
            if (x-returnArray[returnLength-1]) > latencySamplesAllowed:
                returnArray.append(x)
            else:
                removed+=1
    print("Removed: ",removed)
    return returnArray

def checkInput():
    a = input()
    a = int(a)
    if int(a)==0 or int(a)==1:
        return int(a)
    checkInput()

In [123]:
#lookForPossibleChannelSpikes(edfObject,montageArray,stepSize,searchSize,ampThreshold,ptpThreshold,durationToCheck,filterTag=False,loFilter=1,highFilter=40)
possibleSpikesPrelim = lookForPossibleChannelSpikes(trainingEDF1,longBipolarMontage,150,1,2,3,120,filterTag=False,loFilter=1,highFilter=40)

SampleSearchSize:  250 
stepSampleSize:  37 
totalSamples:  30000 
numberOfSteps:  810
StartIndex:  0 
EndIndex:  250
StartIndex:  37 
EndIndex:  287
StartIndex:  74 
EndIndex:  324
StartIndex:  111 
EndIndex:  361
StartIndex:  148 
EndIndex:  398
StartIndex:  185 
EndIndex:  435
StartIndex:  222 
EndIndex:  472
StartIndex:  259 
EndIndex:  509
StartIndex:  296 
EndIndex:  546
StartIndex:  333 
EndIndex:  583
StartIndex:  370 
EndIndex:  620
StartIndex:  407 
EndIndex:  657
StartIndex:  444 
EndIndex:  694
StartIndex:  481 
EndIndex:  731
StartIndex:  518 
EndIndex:  768
StartIndex:  555 
EndIndex:  805
StartIndex:  592 
EndIndex:  842
StartIndex:  629 
EndIndex:  879
StartIndex:  666 
EndIndex:  916
StartIndex:  703 
EndIndex:  953
StartIndex:  740 
EndIndex:  990
StartIndex:  777 
EndIndex:  1027
StartIndex:  814 
EndIndex:  1064
StartIndex:  851 
EndIndex:  1101
StartIndex:  888 
EndIndex:  1138
StartIndex:  925 
EndIndex:  1175
StartIndex:  962 
EndIndex:  1212
StartIndex:  999 
En

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in double_scalars


 19055 
EndIndex:  19305
StartIndex:  19092 
EndIndex:  19342
StartIndex:  19129 
EndIndex:  19379
StartIndex:  19166 
EndIndex:  19416
StartIndex:  19203 
EndIndex:  19453
StartIndex:  19240 
EndIndex:  19490
StartIndex:  19277 
EndIndex:  19527
StartIndex:  19314 
EndIndex:  19564
StartIndex:  19351 
EndIndex:  19601
StartIndex:  19388 
EndIndex:  19638
StartIndex:  19425 
EndIndex:  19675
StartIndex:  19462 
EndIndex:  19712
StartIndex:  19499 
EndIndex:  19749
StartIndex:  19536 
EndIndex:  19786
StartIndex:  19573 
EndIndex:  19823
StartIndex:  19610 
EndIndex:  19860
StartIndex:  19647 
EndIndex:  19897
StartIndex:  19684 
EndIndex:  19934
StartIndex:  19721 
EndIndex:  19971
StartIndex:  19758 
EndIndex:  20008
StartIndex:  19795 
EndIndex:  20045
StartIndex:  19832 
EndIndex:  20082
StartIndex:  19869 
EndIndex:  20119
StartIndex:  19906 
EndIndex:  20156
StartIndex:  19943 
EndIndex:  20193
StartIndex:  19980 
EndIndex:  20230
StartIndex:  20017 
EndIndex:  20267
StartIndex:  

StartIndex:  2812 
EndIndex:  3062
StartIndex:  2849 
EndIndex:  3099
StartIndex:  2886 
EndIndex:  3136
StartIndex:  2923 
EndIndex:  3173
StartIndex:  2960 
EndIndex:  3210
StartIndex:  2997 
EndIndex:  3247
StartIndex:  3034 
EndIndex:  3284
StartIndex:  3071 
EndIndex:  3321
StartIndex:  3108 
EndIndex:  3358
StartIndex:  3145 
EndIndex:  3395
StartIndex:  3182 
EndIndex:  3432
StartIndex:  3219 
EndIndex:  3469
StartIndex:  3256 
EndIndex:  3506
StartIndex:  3293 
EndIndex:  3543
StartIndex:  3330 
EndIndex:  3580
StartIndex:  3367 
EndIndex:  3617
StartIndex:  3404 
EndIndex:  3654
StartIndex:  3441 
EndIndex:  3691
StartIndex:  3478 
EndIndex:  3728
StartIndex:  3515 
EndIndex:  3765
StartIndex:  3552 
EndIndex:  3802
StartIndex:  3589 
EndIndex:  3839
StartIndex:  3626 
EndIndex:  3876
StartIndex:  3663 
EndIndex:  3913
StartIndex:  3700 
EndIndex:  3950
StartIndex:  3737 
EndIndex:  3987
StartIndex:  3774 
EndIndex:  4024
StartIndex:  3811 
EndIndex:  4061
StartIndex:  3848 
E

EndIndex:  16826
StartIndex:  16613 
EndIndex:  16863
StartIndex:  16650 
EndIndex:  16900
StartIndex:  16687 
EndIndex:  16937
StartIndex:  16724 
EndIndex:  16974
StartIndex:  16761 
EndIndex:  17011
StartIndex:  16798 
EndIndex:  17048
StartIndex:  16835 
EndIndex:  17085
StartIndex:  16872 
EndIndex:  17122
StartIndex:  16909 
EndIndex:  17159
StartIndex:  16946 
EndIndex:  17196
StartIndex:  16983 
EndIndex:  17233
StartIndex:  17020 
EndIndex:  17270
StartIndex:  17057 
EndIndex:  17307
StartIndex:  17094 
EndIndex:  17344
StartIndex:  17131 
EndIndex:  17381
StartIndex:  17168 
EndIndex:  17418
StartIndex:  17205 
EndIndex:  17455
StartIndex:  17242 
EndIndex:  17492
StartIndex:  17279 
EndIndex:  17529
StartIndex:  17316 
EndIndex:  17566
StartIndex:  17353 
EndIndex:  17603
StartIndex:  17390 
EndIndex:  17640
StartIndex:  17427 
EndIndex:  17677
StartIndex:  17464 
EndIndex:  17714
StartIndex:  17501 
EndIndex:  17751
StartIndex:  17538 
EndIndex:  17788
StartIndex:  17575 
E

...

EndIndex:  23116
StartIndex:  22903 
EndIndex:  23153
StartIndex:  22940 
EndIndex:  23190
StartIndex:  22977 
EndIndex:  23227
StartIndex:  23014 
EndIndex:  23264
StartIndex:  23051 
EndIndex:  23301
StartIndex:  23088 
EndIndex:  23338
StartIndex:  23125 
EndIndex:  23375
StartIndex:  23162 
EndIndex:  23412
StartIndex:  23199 
EndIndex:  23449
StartIndex:  23236 
EndIndex:  23486
StartIndex:  23273 
EndIndex:  23523
StartIndex:  23310 
EndIndex:  23560
StartIndex:  23347 
EndIndex:  23597
StartIndex:  23384 
EndIndex:  23634
StartIndex:  23421 
EndIndex:  23671
StartIndex:  23458 
EndIndex:  23708
StartIndex:  23495 
EndIndex:  23745
StartIndex:  23532 
EndIndex:  23782
StartIndex:  23569 
EndIndex:  23819
StartIndex:  23606 
EndIndex:  23856
StartIndex:  23643 
EndIndex:  23893
StartIndex:  23680 
EndIndex:  23930
StartIndex:  23717 
EndIndex:  23967
StartIndex:  23754 
EndIndex:  24004
StartIndex:  23791 
EndIndex:  24041
StartIndex:  23828 
EndIndex:  24078
StartIndex:  23865 
E

StartIndex:  6956 
EndIndex:  7206
StartIndex:  6993 
EndIndex:  7243
StartIndex:  7030 
EndIndex:  7280
StartIndex:  7067 
EndIndex:  7317
StartIndex:  7104 
EndIndex:  7354
StartIndex:  7141 
EndIndex:  7391
StartIndex:  7178 
EndIndex:  7428
StartIndex:  7215 
EndIndex:  7465
StartIndex:  7252 
EndIndex:  7502
StartIndex:  7289 
EndIndex:  7539
StartIndex:  7326 
EndIndex:  7576
StartIndex:  7363 
EndIndex:  7613
StartIndex:  7400 
EndIndex:  7650
StartIndex:  7437 
EndIndex:  7687
StartIndex:  7474 
EndIndex:  7724
StartIndex:  7511 
EndIndex:  7761
StartIndex:  7548 
EndIndex:  7798
StartIndex:  7585 
EndIndex:  7835
StartIndex:  7622 
EndIndex:  7872
StartIndex:  7659 
EndIndex:  7909
StartIndex:  7696 
EndIndex:  7946
StartIndex:  7733 
EndIndex:  7983
StartIndex:  7770 
EndIndex:  8020
StartIndex:  7807 
EndIndex:  8057
StartIndex:  7844 
EndIndex:  8094
StartIndex:  7881 
EndIndex:  8131
StartIndex:  7918 
EndIndex:  8168
StartIndex:  7955 
EndIndex:  8205
StartIndex:  7992 
E

StartIndex:  20424 
EndIndex:  20674
StartIndex:  20461 
EndIndex:  20711
StartIndex:  20498 
EndIndex:  20748
StartIndex:  20535 
EndIndex:  20785
StartIndex:  20572 
EndIndex:  20822
StartIndex:  20609 
EndIndex:  20859
StartIndex:  20646 
EndIndex:  20896
StartIndex:  20683 
EndIndex:  20933
StartIndex:  20720 
EndIndex:  20970
StartIndex:  20757 
EndIndex:  21007
StartIndex:  20794 
EndIndex:  21044
StartIndex:  20831 
EndIndex:  21081
StartIndex:  20868 
EndIndex:  21118
StartIndex:  20905 
EndIndex:  21155
StartIndex:  20942 
EndIndex:  21192
StartIndex:  20979 
EndIndex:  21229
StartIndex:  21016 
EndIndex:  21266
StartIndex:  21053 
EndIndex:  21303
StartIndex:  21090 
EndIndex:  21340
StartIndex:  21127 
EndIndex:  21377
StartIndex:  21164 
EndIndex:  21414
StartIndex:  21201 
EndIndex:  21451
StartIndex:  21238 
EndIndex:  21488
StartIndex:  21275 
EndIndex:  21525
StartIndex:  21312 
EndIndex:  21562
StartIndex:  21349 
EndIndex:  21599
StartIndex:  21386 
EndIndex:  21636
S

StartIndex:  4551 
EndIndex:  4801
StartIndex:  4588 
EndIndex:  4838
StartIndex:  4625 
EndIndex:  4875
StartIndex:  4662 
EndIndex:  4912
StartIndex:  4699 
EndIndex:  4949
StartIndex:  4736 
EndIndex:  4986
StartIndex:  4773 
EndIndex:  5023
StartIndex:  4810 
EndIndex:  5060
StartIndex:  4847 
EndIndex:  5097
StartIndex:  4884 
EndIndex:  5134
StartIndex:  4921 
EndIndex:  5171
StartIndex:  4958 
EndIndex:  5208
StartIndex:  4995 
EndIndex:  5245
StartIndex:  5032 
EndIndex:  5282
StartIndex:  5069 
EndIndex:  5319
StartIndex:  5106 
EndIndex:  5356
StartIndex:  5143 
EndIndex:  5393
StartIndex:  5180 
EndIndex:  5430
StartIndex:  5217 
EndIndex:  5467
StartIndex:  5254 
EndIndex:  5504
StartIndex:  5291 
EndIndex:  5541
StartIndex:  5328 
EndIndex:  5578
StartIndex:  5365 
EndIndex:  5615
StartIndex:  5402 
EndIndex:  5652
StartIndex:  5439 
EndIndex:  5689
StartIndex:  5476 
EndIndex:  5726
StartIndex:  5513 
EndIndex:  5763
StartIndex:  5550 
EndIndex:  5800
StartIndex:  5587 
E

StartIndex:  18426 
EndIndex:  18676
StartIndex:  18463 
EndIndex:  18713
StartIndex:  18500 
EndIndex:  18750
StartIndex:  18537 
EndIndex:  18787
StartIndex:  18574 
EndIndex:  18824
StartIndex:  18611 
EndIndex:  18861
StartIndex:  18648 
EndIndex:  18898
StartIndex:  18685 
EndIndex:  18935
StartIndex:  18722 
EndIndex:  18972
StartIndex:  18759 
EndIndex:  19009
StartIndex:  18796 
EndIndex:  19046
StartIndex:  18833 
EndIndex:  19083
StartIndex:  18870 
EndIndex:  19120
StartIndex:  18907 
EndIndex:  19157
StartIndex:  18944 
EndIndex:  19194
StartIndex:  18981 
EndIndex:  19231
StartIndex:  19018 
EndIndex:  19268
StartIndex:  19055 
EndIndex:  19305
StartIndex:  19092 
EndIndex:  19342
StartIndex:  19129 
EndIndex:  19379
StartIndex:  19166 
EndIndex:  19416
StartIndex:  19203 
EndIndex:  19453
StartIndex:  19240 
EndIndex:  19490
StartIndex:  19277 
EndIndex:  19527
StartIndex:  19314 
EndIndex:  19564
StartIndex:  19351 
EndIndex:  19601
StartIndex:  19388 
EndIndex:  19638
S

In [143]:
print(possibleSpikesPrelim.keys())
for x in range(16):
    print(len(possibleSpikesPrelim[x]))
    possibleSpikesPrelim[x]=removeCloseSpikes(possibleSpikesPrelim[x],trainingEDF1,300)
    print(len(possibleSpikesPrelim[x]))

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])
246
Removed:  77
169
265
Removed:  101
164
255
Removed:  96
159
262
Removed:  95
167
233
Removed:  68
165
233
Removed:  74
159
236
Removed:  68
168
249
Removed:  80
169
262
Removed:  99
163
243
Removed:  79
164
251
Removed:  81
170
243
Removed:  72
171
258
Removed:  92
166
289
Removed:  103
186
257
Removed:  89
168
243
Removed:  77
166
